**This notebook is updated daily after the report of Minister of Health in Turkey. If you like the kernel, even though it includes simple visualizations, please upvote. I wish you healthy days.**

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import datetime
from datetime import date, timedelta
import json

%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/corona-virus-test-numbers-in-turkey/turkey_covid.csv')
cities = pd.read_csv('/kaggle/input/corona-virus-test-numbers-in-turkey/city_risks.csv')
json_data = "../input/trcitiesgeojson/tr-cities-utf8.json"
geo_str = json.dumps(json.load(open("../input/trcitiesgeojson/tr-cities-utf8.json", 'r'))) # harita verisi

In [ ]:
import folium
from folium.plugins import FastMarkerCluster

m = folium.Map(location=[39.9208, 35], zoom_start=5.7,tiles="cartodbpositron")

scale = np.linspace(cities["risks"].min() - 1, cities["risks"].max() + 1, 6, dtype=int).tolist()
latitudes = cities['latitudes'].values
longitudes = cities['longitudes'].values
locations = list(zip(latitudes, longitudes))
cpth = folium.Choropleth(
            geo_data=json_data, # harita verisi
            data=cities, # gostermek istedigimiz veri
            columns=['cities', 'risks'], # istenilen sutunlar
            fill_color='BuPu',
            key_on='feature.properties.name',
            legend_name="Number of cases per 100000 people").add_to(m)
    
cpth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], labels=False)
)
folium.LayerControl().add_to(m)
m

In [ ]:
m = folium.Map(location=[39.9208, 35], zoom_start=5.7,tiles="cartodbpositron")
for i in range(cities.shape[0]):
    folium.Circle(
        radius=cities.loc[i, 'risks'] * 100,
        location=[cities.loc[i, 'latitudes'], cities.loc[i, 'longitudes']],
        popup=cities.loc[i, 'cities']+ "-" + str(cities.loc[i, "risks"]), 
        color="#3186cc",
        fill=True,
        fill_color="#3186cc"
).add_to(m)
    
m

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df.isnull().sum()

In [ ]:
df['Active'] = df['Confirmed'] - df['Death'] - df['Recovered']

In [ ]:
values=[df['Active'].iloc[-1], df['Recovered'].iloc[-1], df['Death'].iloc[-1]]
colors = ['gold', 'mediumturquoise', 'darkorange']

fig = go.Figure(data=[go.Pie(labels=['Active','Recovered','Death'],
                             values=values)])
fig.update_traces(hoverinfo='label+percent', textinfo='label+value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.update_layout(title_text='Total Situtation', template='plotly_dark')
fig.show()


In [ ]:
values=[df['Daily_Confirmed'].iloc[-1], df['Daily_Recovered'].iloc[-1], df['Daily_Death'].iloc[-1]]
colors = ['gold', 'mediumturquoise', 'darkorange']

fig = go.Figure(data=[go.Pie(labels=['Confirmed','Recovered','Deaths'],
                             values=values)])
fig.update_traces(hoverinfo='label+percent', textinfo='label+value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.update_layout(title_text='Daily '+ str(df['Date'].iloc[-1].strftime("%Y/%m/%d")) +' Statistics', template='plotly_dark')
fig.show()


In [ ]:
fig = px.bar(df, y='Confirmed',title='Total Confirmed Cases',color_discrete_sequence=['#5DADE2'])
fig.update_layout(template='plotly_dark')
fig.show()

**The reason why there is a strike in 26 November is that Minister of Health decided to release the confirmed cases even though they do not show any symptoms. In the past, they were releasing the cases who has symptoms, therefore the numbers were so low.**

In [ ]:
df['daily_confirmed_mean'] = df['Daily_Confirmed'].rolling(7).mean()
fig = go.Figure()

fig.add_trace(go.Scatter( y=df['Daily_Confirmed'],
                    mode='lines+markers',marker_color='blue',name='Daily Confirmed'))

fig.add_trace(go.Scatter( y=df['daily_confirmed_mean'],
                    mode='lines+markers',marker_color='red',name='7 Day Average'))
fig.update_layout(template='plotly_dark')
fig.show()

In [ ]:
fig = px.bar(df,y='Death',title='Total Death Cases',color_discrete_sequence=['#7D3C98'])
fig.update_layout(template='plotly_dark')
fig.show()

In [ ]:
df['daily_death_mean'] = df['Daily_Death'].rolling(7).mean()
fig = go.Figure()

fig.add_trace(go.Scatter( y=df['Daily_Death'],
                    mode='lines+markers',marker_color='blue',name='Daily Death Cases'))

fig.add_trace(go.Scatter( y=df['daily_death_mean'],
                    mode='lines+markers',marker_color='red',name='7 Day Average'))



fig.update_layout(template='ggplot2')
fig.show()

In [ ]:
fig = px.bar(df,y='Test',title='Test Numbers in Each Day',color_discrete_sequence=['#5D6D7E'])
fig.update_layout(template='ggplot2')
fig.show()

In [ ]:
fig = px.bar(df, y='Recovered',title='Total Recovered Cases',color_discrete_sequence=['#5199FF'])
fig.update_layout(template='plotly_dark')
fig.show()

In [ ]:
df['daily_recovered_mean'] = df['Daily_Recovered'].rolling(7).mean()
fig = go.Figure()

fig.add_trace(go.Scatter( y=df['Daily_Recovered'],
                    mode='lines+markers',marker_color='blue',name='Daily Recovered Cases'))

fig.add_trace(go.Scatter( y=df['daily_recovered_mean'],
                    mode='lines+markers',marker_color='red',name='7 Day Average'))



fig.update_layout(template='ggplot2')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( y=df['Confirmed'],
                    mode='lines+markers',marker_color='blue',name='Total Cases'))

fig.add_trace(go.Scatter( y=df['Recovered'],
                    mode='lines+markers',marker_color='green',name='Recovered Cases'))

fig.add_trace(go.Scatter( y=df['Death'],
                    mode='lines+markers',marker_color='red',name='Deaths Cases'))

fig.add_trace(go.Scatter( y=df['Active'],
                    mode='lines+markers',marker_color='#17202A',name='Active Cases'))

fig.update_layout(title_text='Corona Virus Statistics in Turkey', template='ggplot2')
fig.show()



In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( y=df['Daily_Confirmed'],
                    mode='lines+markers',marker_color='blue',name='Daily New Cases'))

fig.add_trace(go.Scatter( y=df['Daily_Death'],
                    mode='lines+markers',marker_color='red',name='Daily Deaths'))

fig.add_trace(go.Scatter( y=df['Daily_Recovered'],
                    mode='lines+markers',marker_color='green',name='Daily Recovered'))
fig.update_layout(title_text='Daily Statistics', template='ggplot2')

fig.show()

In [ ]:

df['Mortality Rate'] = (df['Death'] / df['Confirmed']) * 100
df['Recovery Rate'] = (df['Recovered'] / df['Confirmed']) * 100
df['Spread Rate'] = (df['Daily_Confirmed'] / df['Confirmed'].shift(1)) * 100
df['Positive Rate'] = (df['Daily_Confirmed'] / df['Test'])  * 100


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( y=df['Spread Rate'],
                    mode='lines+markers',marker_color='#1C32AE',name='Spread Rate'))
fig.update_layout(title_text='Spread Rate in Turkey', template='ggplot2')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( y=df['Mortality Rate'],
                    mode='lines+markers',marker_color='green',name='Mortality Rate'))
fig.update_layout(title_text='Mortality Rate in Turkey', template='ggplot2')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( y=df['Recovery Rate'],
                    mode='lines+markers',marker_color='#B96BAA',name='Recovery Rate'))
fig.update_layout(title_text='Recovery Rate in Turkey', template='ggplot2')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( y=df['Positive Rate'],
                    mode='lines+markers',marker_color='#FF005C',name='Positive Test Rate'))
fig.update_layout(title_text='Positive Rate in Turkey', template='ggplot2')
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( y=df['Spread Rate'],
                    mode='lines+markers',marker_color='pink',name='Spread Rate'))

fig.add_trace(go.Scatter( y=df['Mortality Rate'],
                    mode='lines+markers',marker_color='red',name='Mortality Rate'))

fig.add_trace(go.Scatter(y=df['Recovery Rate'],
                    mode='lines+markers',marker_color='green',name='Recovery Rate'))

fig.add_trace(go.Scatter( y=df['Positive Rate'],
                    mode='lines+markers',marker_color='blue',name='Positive Rate'))
fig.update_layout(title_text='Rate Statistics', template='ggplot2')

fig.show()